In [1]:
from CMRQuery import CMRQuery

### Initialize the query object, login to Earthdata and set the authentication token

In [2]:
cmr_query = CMRQuery()

cmr_query.login_and_set_token()

### Get the shortnames for all L4 collections containing 'sst' or 'sea surface temperature' keywords

In [8]:
shortnames = cmr_query.query_collections_by_keyword(
                            provider='POCLOUD',
                            keywords=['sst', 'sea surface temperature'],
                            processinglevel='4')

for s in shortnames:
    print(s)

MUR-JPL-L4-GLOB-v4.1
CMC0.1deg-CMC-L4-GLOB-v3.0
MUR25-JPL-L4-GLOB-v04.2
OSTIA-UKMO-L4-GLOB-v2.0
K10_SST-NAVO-L4-GLOB-v01
AVHRR_OI-NCEI-L4-GLOB-v2.1
DMI_OI-DMI-L4-GLOB-v1.0
GAMSSA_28km-ABOM-L4-GLOB-v01
Geo_Polar_Blended-OSPO-L4-GLOB-v1.0
RAMSSA_09km-ABOM-L4-AUS-v01
AVHRR_OI-NCEI-L4-GLOB-v2.0
CMC0.2deg-CMC-L4-GLOB-v2.0
Geo_Polar_Blended_Night-OSPO-L4-GLOB-v1.0
MITgcm_LLC4320_Pre-SWOT_JPL_L4_BassStrait_v1.0
MW_IR_OI-REMSS-L4-GLOB-v5.0
MW_IR_OI-REMSS-L4-GLOB-v5.1
MW_OI-REMSS-L4-GLOB-v5.0
MW_OI-REMSS-L4-GLOB-v5.1
OISST_HR_NRT-GOS-L4-BLK-v2.0
OISST_HR_NRT-GOS-L4-MED-v2.0
OISST_UHR_NRT-GOS-L4-BLK-v2.0
OISST_UHR_NRT-GOS-L4-MED-v2.0
REYNOLDS_NCDC_L4_MONTHLY_V5


### get the granules belonging to each collection found above, within a time range

In [9]:
granule_URLs = cmr_query.query_granules_by_shortname(
                            shortnames=shortnames, 
                            provider="POCLOUD", 
                            temporal_range="2022-07-18T00:00:00Z,2022-07-18T23:59:59Z")

for g in granule_URLs:
    print(g)

s3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20220718090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
s3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20220719090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
s3://podaac-ops-cumulus-protected/CMC0.1deg-CMC-L4-GLOB-v3.0/20220717120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
s3://podaac-ops-cumulus-protected/CMC0.1deg-CMC-L4-GLOB-v3.0/20220718120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
s3://podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20220718090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc
s3://podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20220719090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc
s3://podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-v2.0/20220717120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB-v02.0-fv02.0.nc
s3://podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-v2.0/20220718120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB-v02.0-fv02.0.nc
s3://podaac-ops-cumulus-pr

### clean up and delete token

In [ ]:
cmr_query.delete_token()